In [ ]:
import os
import sys

import numpy as np

from math import exp, log
from scipy.interpolate import interp1d

sys.path.insert(0, os.path.abspath('../'))

In [ ]:
import pygfunction as gt

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
N_1 = 1
N_2 = 2
B_1 = 5
B_2 = 5
H = 100
D = 1
r_b = 0.05

In [ ]:
boreField = gt.boreholes.rectangle_field(N_1=N_1, N_2=N_2, B_1=B_1, B_2=B_2, H=H, D=D, r_b=r_b)

In [ ]:
boreField

In [ ]:
k_s = 2
rho_cp_s = 2000 * 1000
alpha = k_s / rho_cp_s
dt = 100 * 3600.                # Time step
tmax = 10. * 8760. * 3600.      # Maximum time
Nt = 40                         # Number of time steps
ts = H**2/(9.*alpha)            # Bore field characteristic time
ts_years = ts / 3600 / 24 / 365

In [ ]:
lntts = np.arange(-8.5, 3.25, 0.5)

In [ ]:
lntts

In [ ]:
time = np.exp(lntts) * ts

In [ ]:
ts_years

In [ ]:
cimm_g_flux = gt.gfunction.uniform_heat_extraction(boreField, time, alpha, disp=True)

In [ ]:
cimm_g_flux

In [ ]:
cimm_g_bh_wall = gt.gfunction.uniform_temperature(boreField, time, alpha, disp=True)

In [ ]:
def get_glhepro_data(path):
    
    lntts = []
    g = []
    
    with open(path, 'r') as f:
        header = 39
        for line in f:
            if header > 0:
                header -= 1
            else:
                tokens = line.strip().split(' ')
                lntts.append(float(tokens[0]))
                g.append(float(tokens[-1]))
    return lntts, g

In [ ]:
path_glhe_flux = 'C:\\Users\\mmatts\\Desktop\\4x6_flux.dat'
glhe_lntts_flux, glhe_g_flux = get_glhepro_data(path_glhe_flux)

path_glhe_bh_wall = 'C:\\Users\\mmatts\\Desktop\\4x6_temp.dat'
glhe_lntts_bh_wall, glhe_g_bh_wall = get_glhepro_data(path_glhe_bh_wall)

In [ ]:
def plot_vals(list_of_tupels, save_name):
    
    linestyles = ['-', '--', '-', '--', '-', '--']
    markers = ['x', 'o', 'v', '^', 'v', '<']
    
    fig = plt.figure(figsize=(7, 5), dpi=200)
    ax = fig.add_subplot(2, 1, 1)
    
    for idx, tupel in enumerate(list_of_tupels):
        x = tupel[0]
        y = tupel[1]
        label = tupel[2]
        linestyle = linestyles[idx]
        marker = markers[idx]
        plt.plot(x, y, label=label, linestyle=linestyle, marker=marker)

    plt.ylabel('g-function')
    plt.legend()
    
    ax = fig.add_subplot(2, 1, 2)
    y_1 = np.array(list_of_tupels[0][1])
    
    for idx, tupel in enumerate(list_of_tupels):
        if idx == 0:
            pass
        else:
            x = tupel[0]
            y = y_1 - tupel[1]
            label = tupel[2]
            linestyle = linestyles[idx]
            marker = markers[idx]
            plt.plot(x, y, linestyle=linestyle, marker=marker)
    
    plt.ylabel('g-error\n[GLHEPro - pygfunc]')
    plt.xlabel('ln(t/$t_s$)')
    plt.savefig(save_name, bbox_inches='tight')
    plt.show()

In [ ]:
def normalize_glhepro_data(input_tuple, norm_lntts):
    f_interp = interp1d(input_tuple[0], input_tuple[1])
    label = input_tuple[2]
    g_out = []
    for lntts in norm_lntts:
        g_out.append(f_interp(lntts))
        
    return norm_lntts, g_out, label

In [ ]:
plot_vals([normalize_glhepro_data((glhe_lntts_flux, glhe_g_flux, 'GLHEPro: Uniform Flux'), lntts), 
           (lntts, cimm_g_flux, 'pygfunction: Uniform Flux')],
          "Flux.png")

In [ ]:
plot_vals([normalize_glhepro_data((glhe_lntts_bh_wall, glhe_g_bh_wall, 'GLHEPro: Uniform Wall Temp'), lntts), 
           (lntts, cimm_g_bh_wall, 'pygfunction: Uniform Wall Temp')],
           'BH_wall.png')